# How to use chat models to call tools

Tool calling allows a chat model to respond to a given prompt by "calling a tool".

Remember, while the name "tool calling" implies that the model is directly performing some action, this is actually not the case! The model only generates the arguments to a tool, and actually running the tool(or not) is up to the user.

Tool calling is a general technique that generates structured output from a model, and you can use it even when you don't intend to invoke any tools. An example use-case of that is extraction from unstructured text.

![img1](../asset/tool_call.png)

LangChain implements standard interface for defining tools, passing them to LLMs, and representing tool calls. This guide will cover how to bind tools to an LLM, then invoke the LLM to generate these arguments.

## Defining tool schemas

For a model to be able to call tools, we need to pass in tool schemas that describe what the tool does and what it's arguments are. Chat models that support tool calling features implement a `.bind_tools()` method for passing tool schemas to the model. Tool schemas can be passed in as Python functions(with typehints and docstrings), Pydantic models, TypeDict classes, or LangChain Tool objects. Subsequent invocations of the model will pass in these tool schemas along with the prompt.



## Python functions

Our tool schemas can be Python functions:

In [ ]:
# The function name, type hints, and docstring are all part of the tool
# schema that's passed to the model. Defining good, descriptive schemas
# is an extension of prompt engineering and is an important part of 
# getting models to perform well.

def add(a: int, b: int) -> int:
    """ Add two integers.
    
    Args:
        a: First integer.
        b: Second integer.
    """
    return a + b

def multiply(a: int, b: int) -> int:
    """Multiply two integers.
    
    Args:
        a: First integer.
        b: Second integer. 
    """
    return a * b

## LangChain Tool

LangChain also implements a `@tool` decorator that allows for further control of the tool schema, such as tool names and argument descriptions.

## Pydantic class

You can equivalently define the schemas without the accompanying functions using Pydantic.

Note that all fields are `required` unless provided a default value.





In [2]:
from pydantic import BaseModel, Field

class add(BaseModel):
    """Add two integers."""
    
    a: int = Field(..., description="First integer.")
    b: int = Field(..., description="Second integer.")
    
class multiply(BaseModel):
    """Multiply two integers."""
    
    a: int = Field(..., description="First integer.")
    b: int = Field(..., description="Second integer.")

## TypedDict class

Or using TypedDicts and annotations:

In [3]:
from typing_extensions import Annotated, TypedDict

class add(TypedDict):
    """Add two integers."""
    
    # Annotations must have the type and can optionally include a default value and description
    # (in that order)
    
    a: Annotated[int, ...,'First integer.']
    b: Annotated[int, ...,'Second integer.']
    
class multiply(BaseModel):
    """Multiply two integers."""
    
    a: Annotated[int, ...,'First integer.']
    b: Annotated[int, ...,'Second integer.']
    
tools = [add, multiply]

To actually bind those schemas to a chat model, we'll use the `.bind_tools()` method. This handles converting the `add` and `multiply` schemas to the proper format for the model. The tool schema will then be passed it in each time the model is invoked.

In [4]:
import os
import yaml
from langchain_community.chat_models import ChatZhipuAI


with open('../utils/config.yml', 'r') as stream:
    zhipuai_api_key = yaml.safe_load(stream)['api_key']

os.environ['ZHIPUAI_API_KEY'] = zhipuai_api_key

llm = ChatZhipuAI(
    model='glm-4-plus',
    temperature=0,
)

In [5]:
llm_with_tools = llm.bind_tools(tools)
query = "what's 3 * 12 ?"
llm_with_tools.invoke(query)

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'id': 'call_9021294753560307203', 'index': 0, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 285, 'total_tokens': 300}, 'model_name': 'glm-4-plus', 'finish_reason': 'tool_calls'}, id='run-0a9c864c-bbf1-4073-b982-c9aac7b8d50a-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_9021294753560307203', 'type': 'tool_call'}])

As we can see our LLM generated arguments to a tool! You can look at the docs for bind_tools() to learn about all the ways to customize how your LLM selects tools, as well as this guide on how to force the LLM to call a tool rather than letting it decide.

## Tool calls

If tool calls are included in a LLM response, they are attached to the corresponding message or message chunk as a list of tool call objects in the `.tool_calls` attribute.

Note that chat models can call multiple tools at once.

A `ToolCall` is typed dict that includes a tool name, dict of argument values, and(optionally) and identifier. Message with no tool calls default to an empty list for this attribute.

In [6]:
query = "what's 3 * 12 ? Also, what is 11 + 49?"
llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_9021299563924013477',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_9021299563924013478',
  'type': 'tool_call'}]

The `.tool_calls` attribute should contain valid tool calls. Note that on occasion, model provides may output malformed tool calls(e.g.,arguments that are not valid JSON). When parsing fails in these cases, instances of InvalidToolCall are populated in the `.invalid_tool_calls` attribute. An `InvalidToolCall` can have a name, string arguments, identifier, and error message.

## Parsing

If desired, output parsers can further process the output. For example, we can convert existing values populated on the `tool_calls` to Pydantic objects using the PydanticToolsParser:

In [7]:
from langchain_core.output_parsers import PydanticToolsParser
from pydantic import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


chain = llm_with_tools | PydanticToolsParser(tools=[add, multiply])
chain.invoke(query)

[multiply(a=3, b=12), add(a=11, b=49)]